### Imports

In [51]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import timeit
from tqdm.notebook import tqdm

## make sure to run this script from the root of the repository
# or adjust the sys.path accordingly to find the neck_brace_kinematics module
import sys
import os
sys.path.append(os.path.abspath('../..'))

from neck_brace_kinematics import forward_kinematics, inverse_kinematics, evaluate_ee_fk, C_func, C_star_func
from visualize_robot import visualize_robot

## Sweeping through End-effector Poses

In [63]:
# Define sweep ranges
alpha_range = np.linspace(np.deg2rad(-41), np.deg2rad(41), 25)  # 100 points from -41 to 41 degrees
beta_range = np.linspace(np.deg2rad(-37), np.deg2rad(80), 25)
gamma_range = np.linspace(np.deg2rad(-30), np.deg2rad(30), 25)

# Create empty arrays to store the results
num_points = len(alpha_range) * len(beta_range) * len(gamma_range)
poses = np.zeros((num_points, 4))
ik_results = np.zeros((num_points, 12))
Cs = np.zeros((num_points, 12, 4))  
C_star_invs = np.zeros((num_points, 12, 12))

In [64]:

idx = 0
initial_guess = [0.2, 0.2, 0.2, 1.75, np.deg2rad(100), 0, 0, 0, 0, np.deg2rad(80), 0, 0]

pbar = tqdm(total=num_points, desc="Computing IK", unit="pose")
for alpha in alpha_range:
    for beta in beta_range:
        for gamma in gamma_range:
            pose = np.array([alpha, beta, gamma, 0.150])
            poses[idx, :] = pose
            
            ik_result = inverse_kinematics(*pose, initial_guess=initial_guess)
            initial_guess = ik_result  # Update initial guess for next iteration

            ik_results[idx, :] = ik_result
            idx += 1
            pbar.update(1)

pbar = tqdm(total=num_points, desc="Computing Jacobians", unit="pose")
for i in range(num_points):
    q = np.concatenate((ik_results[i, :], poses[i, :]))
    C = C_func(*q)
    C_star = C_star_func(*q)
    Cs[i] = C
    C_star_invs[i] = np.linalg.pinv(C_star)
    pbar.update(1)


Computing IK:   0%|          | 0/15625 [00:00<?, ?pose/s]

Computing Jacobians:   0%|          | 0/15625 [00:00<?, ?pose/s]

In [65]:
# Create an array to store the average positions of E1 and E3
avg_positions = np.zeros((num_points, 3))

# Calculate the end effector positions and average of E1 and E3
for i in range(num_points):
    E1_ee_val, E2_ee_val, E3_ee_val, A_val = evaluate_ee_fk(*poses[i])
    
    # Convert to numpy arrays and flatten
    E1_ee_eval = E1_ee_val.flatten().astype(np.float64)
    E3_ee_eval = E3_ee_val.flatten().astype(np.float64)
    
    # Calculate average position of E1 and E3
    avg_positions[i] = 0.5 * (E1_ee_eval + E3_ee_eval)

# Compute condition numbers
condition_numbers = np.zeros(num_points)
pbar = tqdm(total=num_points, desc="Computing Condition Numbers", unit="pose")
for i in range(num_points):
    C = Cs[i]
    C_star_inv = C_star_invs[i]
    
    # Calculate condition number as the product of C* inverse and C
    condition_numbers[i] = np.linalg.cond(C_star_inv @ C)
    pbar.update(1)

condition_numbers = np.clip(condition_numbers, None, 100)

Computing Condition Numbers:   0%|          | 0/15625 [00:00<?, ?pose/s]

In [66]:
import plotly.graph_objects as go

home_pose = [0, 0, 0, 0.150]  # Define the home pose for the end effector
ik_home = inverse_kinematics(*home_pose)

# Create a scatter3d trace for the average positions
fig = visualize_robot(*home_pose, *ik_home, width=None, height=None, showlegend=False)

# Add the scatter points with color based on distance from z-axis
fig.add_trace(go.Scatter3d(
    x=avg_positions[:, 0],
    y=avg_positions[:, 1],
    z=avg_positions[:, 2],
    mode='markers',
    marker=dict(
        size=5,
        color= condition_numbers, # Use condition numbers for color
        colorscale='jet',
        opacity=0.2,
        colorbar=dict(title='Condition Number')
    ),
    name='End Effector Positions'
))

# Set the layout
fig.update_layout(
    margin=dict(l=0, r=50, b=0, t=30),
    width=800,
    height=600
)

# Show the figure
fig.show()

# Save the figure as an HTML file
fig.write_html("condition_number_visualization.html")